This notebook satisfies the environmental requirements of running cGAN experiments in Colab while monitoring via TensorBoard.

We start off by making Google Drive files visible to this notebook.
We then attach TensorBoard and have it monitor an output directory (LOG_DIR).

Finally we incorporate an experimental section from which tests can be run.


# Make Google Drive files accessible

See https://colab.research.google.com/notebooks/io.ipynb#scrollTo=jRQ5_yMcqJiV for more info.

This will prompt you to open a new tab that allows Colab access to your Google Drive.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!ls '/content/gdrive/'

## UNFORTUNATELY, we cannot mount the filesystem
## which ought to be rooted in "Shared with me"--i.e. shared files.
##
## The user will need to copy whatever datasets and imports are needed
## and set up directory paths directly.

'My Drive'  'Team Drives'


In [0]:
# OPTIONAL:
# Prove that the mountpoint works. Only needs to be run once.
# Also it will overwrite anything in your google drive home directory that
# was named foo.txt, so maybe don't run it if that's like a wallet with sixty
# bitcoins in it or something
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive! I am Colab!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive! I am Colab!

# Set Up Global Parameters
This includes command-line arguments and imports.

Note that because a Jupyter notebook, particularly in the cloud environment,
does not have a command line per se, flag values are passed as strings.

In [ ]:
#!/usr/bin/python3


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import time
import functools
from six.moves import xrange  # pylint: disable=redefined-builtin

import tensorflow as tf
import argparse
import os
import json
import glob
import random
import collections
import math
import time
import sys

# SET THESE VALUES TO THE PATHS THAT CORRECTLY DESCRIBE YOUR ENVIRONMENT!
DRIVE_BASE_DIR = '/content/gdrive/My Drive/2018-Fa Drori Intro ML/Group Project/'
HANDBAG_DIR = DRIVE_BASE_DIR + 'handbags/500_bags'
FACADE_DIR = DRIVE_BASE_DIR + 'facades/train'
## ASSUME THAT "SLIM" DIRECTORY AND "NETWORKS.PY" FROM GITHUB
## ARE STORED SOMEWHERE UNDER LIBDIR
LIBDIR = DRIVE_BASE_DIR + 'imports'

# Update our includes-path to import other files stored in Drive
# Obviously LIBDIR must be defined first!
if not os.path.isdir(LIBDIR): raise Exception("LIBDIR not set.")
if not LIBDIR in sys.path:
  sys.path.append(LIBDIR)
if not os.path.isdir(HANDBAG_DIR): raise Exception("Default data path not set.")
if not os.path.isdir(FACADE_DIR): raise Exception("Default data path not set.")
    
import networks
from slim.nets import cyclegan
from slim.nets import pix2pix



# Main TFGAN library.
tfgan = tf.contrib.gan
CROP_SIZE = 128
DEFAULT_SCALE = 140

parser = argparse.ArgumentParser()

#added default argument
parser.add_argument("--input_dir",default="", help="path to folder containing images")
parser.add_argument("--output_dir",default="", help="where to put output files")

parser.add_argument("--max_steps", default=1000,type=int,
                    help="max number of training steps (0 to disable)")
parser.add_argument("--max_epochs", type=int, help="max number of training epochs")
parser.add_argument("--summary_freq", type=int, default=1,
                    help="update summaries every summary_freq steps")
parser.add_argument("--progress_freq", type=int, default=1,
                    help="display progress every progress_freq steps")
parser.add_argument("--trace_freq", type=int, default=1,
                    help="trace execution every trace_freq steps")
parser.add_argument("--display_freq", type=int, default=1,
                    help="write current training images every display_freq steps")
parser.add_argument("--save_freq", type=int, default=5000,
                    help="save model every save_freq steps, 0 to disable")

### IMPORTANT PARAMETER ARGUMENTS
parser.add_argument("--scale_size", type=int, default=DEFAULT_SCALE,
                    help="scale images to this size before cropping to CROPSIZE x CROPSIZE")
parser.add_argument("--batch_size", type=int, default=2,
                    help="number of images in batch")
### to be used for extension to pix2pix-style loss function--presently unused
parser.add_argument("--l1_weight", type=float, default=100.0, help="weight on L1 term for generator gradient")
parser.add_argument("--gan_weight", type=float, default=1.0, help="weight on GAN term for generator gradient")

parser.add_argument("--n_critic", type=int, default=5, help="Number of training iterations for the discriminator for each generator iteration")

parser.add_argument("--loss_fn", choices=['wgan', 'mod', 'minimax', 'pix2pix'],
                    default='wgan', help="loss function to use for generator and discriminator.")
parser.add_argument("--disc_alpha", type=float, default=0.0001, help="Learning rate for discriminator network")
parser.add_argument("--gen_alpha", type=float, default=0.001, help="Learning rate for generator network")
parser.add_argument("--dataset", choices=['handbags', 'facades'],
                    default='handbags', help="Choose dataset to use")

a = parser.parse_args(args=["--n_critic=3", "--max_steps=1000",
                            "--loss_fn=wgan",
                            #"--disc_alpha=0.0001",
                            "--dataset=handbags",
                            "--batch_size=1" ])
if a.output_dir == "":
  a.output_dir = "{}-{}-ncrit{}-batch{}-alpha{}-{}".format(a.dataset, a.max_steps, a.n_critic,
                                                           a.batch_size, a.disc_alpha, a.loss_fn)
  
if a.dataset == 'handbags':
  a.input_dir = HANDBAG_DIR
elif a.dataset == 'facades':
  a.input_dir = FACADE_DIR
else:
  raise ValueError("Unknown dataset.")

print("Input: {} Output: {}".format(a.input_dir, a.output_dir))


# Connect TensorBoard to Colab

In [0]:
### Here's how you'd do it with ngrok, just here for reference.
### We used localtunnel.
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip


In [0]:
#import time
#get_ipython().system_raw('./ngrok http 6006 &')
#time.sleep(2)

#! curl -s http://localhost:4040/api/tunnels | python3 -c \
#    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
## Install requirements
# (Only need to execute this cell once)
! npm i -g npm
! npm install -g localtunnel

/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
+ npm@6.4.1
added 273 packages from 152 contributors, removed 419 packages and updated 40 packages in 13.862s
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
added 54 packages from 31 contributors in 2.45s


In [0]:
import time
# Now launch TensorBoard pointing to the globally configured log directory.
print("Executing: tensorboard --logdir {} --host 0.0.0.0 --port 6006 &"
  .format(a.output_dir))
# Shell out to execute tensorboard
get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(a.output_dir)
)

time.sleep(2)

# Tunnel port 6006 (assume TensorBoard ran successfully)
get_ipython().system_raw('lt --port 6006 > url.txt 2>&1 &')

time.sleep(2)

print ("Connect to TensorBoard from wherever at this URL:")
! cat url.txt

Executing: tensorboard --logdir /content/gdrive/My Drive/Group Project/log-fullRun/ --host 0.0.0.0 --port 6006 &
Connect to TensorBoard from wherever at this URL:
your url is: https://hot-grasshopper-20.localtunnel.me


If you change the logging directory, you'll need to kill
the existing TensorBoard process and start
a new one. (So far as I can tell, restarting won't succeed; there can only be one
process listening on a port at a time anyway.)

The following cell will do that (you need to run it, look at the output, then modify the "kill" line and uncomment it so it will actually kill the process.) The rerun of ps is a double-check that you did successfully shut TensorBoard down.


In [0]:
## RUN THIS CELL ONLY IF YOU NEED TO RESTART TENSORBOARD (like to point to a new log dir)
import time
!ps | head -n 1
!ps | grep 'tensorboard'
## Read the "PID" column to figure out what process number TensorBoard is,
## then use that processs in the next line
# !kill PID_GOES_HERE
time.sleep(2)
!ps | grep 'tensorboard'

    PID TTY          TIME CMD
    291 ?        00:02:54 tensorboard
    291 ?        00:02:55 tensorboard


In [0]:
# Run this ONLY to clear out old log files.
# Commented out the actual deleters to avoid accidental execution.
import os, shutil
for the_file in os.listdir(LOG_DIR):
    file_path = os.path.join(LOG_DIR, the_file)
    try:
        if os.path.isfile(file_path):
            print("Unlinking {}".format(file_path))
#            os.unlink(file_path)
        elif os.path.isdir(file_path):
            print ("Recursively removing {}".format(file_path))
#            shutil.rmtree(file_path)
    except Exception as e:
        print(e)

Unlinking /content/gdrive/My Drive/Group Project/log-fullRun/graph.pbtxt
Recursively removing /content/gdrive/My Drive/Group Project/log-fullRun/images
Unlinking /content/gdrive/My Drive/Group Project/log-fullRun/index.html
Unlinking /content/gdrive/My Drive/Group Project/log-fullRun/events.out.tfevents.1543241235.59feb57c6f7e
Unlinking /content/gdrive/My Drive/Group Project/log-fullRun/events.out.tfevents.1543271338.943eaeaf319f
Unlinking /content/gdrive/My Drive/Group Project/log-fullRun/events.out.tfevents.1543271566.943eaeaf319f
Unlinking /content/gdrive/My Drive/Group Project/log-fullRun/events.out.tfevents.1543273566.943eaeaf319f


## Test It

Only need to execute this to prove that the tensorboard connection is working.

If you do run this, be sure to clean out LOG_DIR afterward.

In [0]:
#### EXAMPLE JUST TO PROVE IT'S WORKING
### DON'T RUN THIS UNLESS YOU JUST WANT TO TEST TENSORBOARD-IN-THE-CLOUD
### IF YOU DO, RUN THE "CLEAN OUT LOG DIR" CELL AFTERWARD

from __future__ import print_function

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1

x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

with tf.name_scope('Model'):
    pred = tf.nn.softmax(tf.matmul(x,W) + b)
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

init = tf.global_variables_initializer()

tf.summary.scalar("loss", cost)
tf.summary.scalar("accuracy", acc)
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)

    summary_writer = tf.summary.FileWriter(LOG_DIR, graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # optimize/backprop, do cost (compute loss), summarize
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary, epoch * total_batch + i)
            avg_cost += c / total_batch
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=",
                  "{:.9f}".format(avg_cost))

    print ("Training done!")
    print("Accuracy:", acc.eval({x: mnist.test.images, y:mnist.test.labels}))
    print("Now try connecting to the address from the previous cell.")


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

# Experiment Code

This is the main code for whatever you want to test!
If you have used a custom argument parser above, you could use
whatever experiment you want to run down below.

In [ ]:
Examples = collections.namedtuple("Examples", "paths, inputs, targets, count, "
                                              "steps_per_epoch")

GEN_LOSS = None
DISC_LOSS = None
if (a.loss_fn == 'wgan'):
  print("Chose wasserstein loss")
  GEN_LOSS = tfgan.losses.wasserstein_generator_loss
  DISC_LOSS = tfgan.losses.wasserstein_discriminator_loss
elif (a.loss_fn == 'mod'):
  print("Chose modified loss")
  GEN_LOSS = tfgan.losses.modified_generator_loss
  DISC_LOSS = tfgan.losses.modified_discriminator_loss
elif (a.loss_fn == 'minimax'):
  print("Chose minimax loss")
  GEN_LOSS = tfgan.losses.minimax_generator_loss
  DISC_LOSS = tfgan.losses.minimax_discriminator_loss
elif (a.loss_fn == 'pix2pix'):
  raise Exception("Full pix2pix loss function not implemented.")
else:
  raise ValueError("Impossible loss function set.")


def append_index(filesets, step=False):
    index_path = os.path.join(a.output_dir, "index.html")
    if os.path.exists(index_path):
        index = open(index_path, "a")
    else:
        index = open(index_path, "w")
        index.write("<html><body><table><tr>")
        if step:
            index.write("<th>step</th>")
        index.write("<th>name</th><th>input</th><th>output</th><th>target</th></tr>")

    for fileset in filesets:
        index.write("<tr>")

        if step:
            index.write("<td>%d</td>" % fileset["step"])
        index.write("<td>%s</td>" % fileset["name"])

        for kind in ["inputs", "outputs", "targets"]:
            index.write("<td><img src='images/%s'></td>" % fileset[kind])

        index.write("</tr>")
    return index_path


def preprocess(image):
    with tf.name_scope("preprocess"):
        # [0, 1] => [-1, 1]
        return image * 2 - 1


def deprocess(image):
    with tf.name_scope("deprocess"):
        # [-1, 1] => [0, 1]
        return (image + 1) / 2


def save_images(fetches, step=None):
    image_dir = os.path.join(a.output_dir, "images")
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

    filesets = []
    for i, in_path in enumerate(fetches["paths"]):
        name, _ = os.path.splitext(os.path.basename(in_path.decode("utf8")))
        fileset = {"name": name, "step": step}
        description = { "inputs": "edge", "targets": "bag", "outputs": "generated" }
        for kind in ["targets", "inputs", "outputs"]:
            filename = name + "-" + description[kind] + ".png"
            if step is not None:
                filename = "%08d-%s" % (step, filename)
            fileset[kind] = filename
            out_path = os.path.join(image_dir, filename)
            contents = fetches[kind][i]
            with open(out_path, "wb") as f:
                f.write(contents)
        filesets.append(fileset)
    return filesets

def load_examples():
    if a.input_dir is None or not os.path.exists(a.input_dir):
        raise Exception("input_dir does not exist")

    input_paths = glob.glob(os.path.join(a.input_dir, "*.jpg"))
    decode = tf.image.decode_jpeg
    if len(input_paths) == 0:
        input_paths = glob.glob(os.path.join(a.input_dir, "*.png"))
        decode = tf.image.decode_png

    if len(input_paths) == 0:
        raise Exception("input_dir contains no image files")

    def get_name(path):
        name, _ = os.path.splitext(os.path.basename(path))
        return name

    # if the image names are numbers, sort by the value rather than asciibetically
    # having sorted inputs means that the outputs are sorted in test mode
    if all(get_name(path).isdigit() for path in input_paths):
        input_paths = sorted(input_paths, key=lambda path: int(get_name(path)))
    else:
        input_paths = sorted(input_paths)

    with tf.name_scope("load_images"):
        # we're always training
        path_queue = tf.train.string_input_producer(input_paths, shuffle=True)
        reader = tf.WholeFileReader()
        paths, contents = reader.read(path_queue)
        raw_input = decode(contents)
        raw_input = tf.image.convert_image_dtype(raw_input, dtype=tf.float32)

        assertion = tf.assert_equal(tf.shape(raw_input)[2], 3, message="image does not have 3 channels")
        with tf.control_dependencies([assertion]):
            raw_input = tf.identity(raw_input)

        raw_input.set_shape([None, None, 3])

        # break apart image pair and move to range [-1, 1]
        width = tf.shape(raw_input)[1] # [height, width, channels]
        left_images = preprocess(raw_input[:,:width//2,:])
        right_images = preprocess(raw_input[:,width//2:,:])

    if (a.dataset == 'handbags'):
      inputs, targets = [left_images, right_images]
    elif (a.dataset == 'facades'):
      targets, inputs = [left_images, right_images]
    else:
      raise ValueError("Unknown dataset.")

    # synchronize seed for image operations so that we do the same operations to both
    # input and output images
    seed = random.randint(0, 2**31 - 1)
    def transform(image):
        r = image
        # area produces a nice downscaling, but does nearest neighbor for upscaling
        # assume we're going to be doing downscaling here
        r = tf.image.resize_images(r, [a.scale_size, a.scale_size], method=tf.image.ResizeMethod.AREA)

        offset = tf.cast(tf.floor(tf.random_uniform([2], 0, a.scale_size - CROP_SIZE + 1, seed=seed)), dtype=tf.int32)
        if a.scale_size > CROP_SIZE:
            r = tf.image.crop_to_bounding_box(r, offset[0], offset[1], CROP_SIZE, CROP_SIZE)
        elif a.scale_size < CROP_SIZE:
            raise Exception("scale size cannot be less than crop size")
        return r

    with tf.name_scope("input_images"):
        input_images = transform(inputs)

    with tf.name_scope("target_images"):
        target_images = transform(targets)

    paths_batch, inputs_batch, targets_batch = tf.train.batch([paths, input_images, target_images], batch_size=a.batch_size)
    steps_per_epoch = int(math.ceil(len(input_paths) / a.batch_size))

    return Examples(
        paths=paths_batch,
        inputs=inputs_batch,
        targets=targets_batch,
        count=len(input_paths),
        steps_per_epoch=steps_per_epoch,
    )


def main():
    if not os.path.exists(a.output_dir):
        os.makedirs(a.output_dir)
    examples = load_examples()
    inputs=examples.inputs
    targets=examples.targets

    gan_model = tfgan.gan_model(
        generator_fn=networks.generator,
        discriminator_fn=networks.discriminator,
        real_data=targets,
        generator_inputs=inputs)

    outputs=gan_model.generated_data

    with tf.name_scope('losses'):
      gan_loss = tfgan.gan_loss(
          gan_model,
          generator_loss_fn=tfgan.losses.modified_generator_loss,
          discriminator_loss_fn=tfgan.losses.modified_discriminator_loss)

    with tf.name_scope('gan_train_ops'):
        generator_optimizer = tf.train.RMSPropOptimizer(a.gen_alpha)
        discriminator_optimizer = tf.train.RMSPropOptimizer(a.disc_alpha)
        gan_train_ops = tfgan.gan_train_ops(
            gan_model,
            gan_loss,
            generator_optimizer,
            discriminator_optimizer)

    steps = tfgan.GANTrainSteps(
        generator_train_steps=1,
        discriminator_train_steps=a.n_critic)



    def convert(image):
        return tf.image.convert_image_dtype(image, dtype=tf.uint8, saturate=True)

    with tf.name_scope("convert_inputs"):
        converted_inputs = convert(inputs)

    with tf.name_scope("convert_targets"):
        converted_targets = convert(targets)

    with tf.name_scope("convert_outputs"):
        converted_outputs = convert(outputs)

        display_fetches = {
            "paths": examples.paths,
            "inputs": tf.map_fn(tf.image.encode_png, converted_inputs, dtype=tf.string, name="input_pngs"),
            "targets": tf.map_fn(tf.image.encode_png, converted_targets, dtype=tf.string, name="target_pngs"),
            "outputs": tf.map_fn(tf.image.encode_png, converted_outputs, dtype=tf.string, name="output_pngs"),
        }

    # summaries
    with tf.name_scope("inputs_summary"):
        tf.summary.image("inputs", converted_inputs)

    with tf.name_scope("targets_summary"):
        tf.summary.image("targets", converted_targets)

    with tf.name_scope("outputs_summary"):
        tf.summary.image("outputs", converted_outputs)

    generator_loss,discriminator_loss=gan_loss
    tf.summary.scalar("discriminator_loss", discriminator_loss)
    tf.summary.scalar("generator_loss_GAN", generator_loss)

    train_step_fn = tfgan.get_sequential_train_steps(train_steps=steps)
    global_step = tf.train.get_or_create_global_step()


    logdir = a.output_dir if (a.trace_freq > 0 or a.summary_freq > 0) else None
    sv = tf.train.Supervisor(logdir=logdir, save_summaries_secs=0, saver=None)

    with sv.managed_session() as sess:
        if a.max_epochs is not None:
            max_steps = examples.steps_per_epoch * a.max_epochs
        if a.max_steps is not None:
            max_steps = a.max_steps


        start=time.time()
        for step in range(max_steps):
            print('step=',step)
            def should(freq):
                return freq > 0 and ((step + 1) % freq == 0 or step == max_steps - 1)

            options = None
            run_metadata = None
            if should(a.trace_freq):
                options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
            print('running train step')
            curr_loss, glo_gang = train_step_fn(
                sess, gan_train_ops, global_step, train_step_kwargs={})
            print('curr_loss:',curr_loss)
            print('glo_gang:',glo_gang)
            print('ran train step')
            fetches = { }

            if should(a.progress_freq):
                fetches["discrim_loss"],fetches['gen_loss'] = gan_loss

            if should(a.summary_freq):
                fetches["summary"] = sv.summary_op

            if should(a.display_freq):
                fetches["display"] = display_fetches

            # values in fetches are the variables that get computed by sess
            print('running sess.run')
            results = sess.run(fetches, options=options, run_metadata=run_metadata)
            print('ran sess.run')
            if should(a.summary_freq):
                print("recording summary")
                sv.summary_writer.add_summary(results["summary"], step)

            if should(a.display_freq):
                print("saving display images")
                filesets = save_images(results["display"], step=step+1)
                append_index(filesets, step=True)

            if should(a.trace_freq):
                print("recording trace")
                sv.summary_writer.add_run_metadata(run_metadata, "step_%d" % step)

            if should(a.progress_freq):
                # global_step will have the correct step count if we resume from a checkpoint
                train_epoch = math.ceil(step / examples.steps_per_epoch)
                train_step = (step - 1) % examples.steps_per_epoch + 1
                rate = (step + 1) * a.batch_size / (time.time() - start)
                remaining = (max_steps - step) * a.batch_size / rate
                print("progress  epoch %d  step %d  image/sec %0.1f  remaining %dm" % (
                train_epoch, train_step, rate, remaining / 60))
                print("discrim_loss", results["discrim_loss"])
                print("gen_loss", results["gen_loss"])

            # We aren't actually trying to output a reusable model.
            # if should(a.save_freq):
            #     print("saving model")
            #     saver.save(sess, os.path.join(a.output_dir, "model"), global_step=sv.global_step)

if __name__=='__main__':
    main()
